## Data Preprocessing

This notebook contains code for data cleaning and ratio computation over the original [data set](https://osf.io/xsb6g/) and produces a CSV file used in our main analyses.

In [1]:
import pandas as pd
import numpy as np

In [2]:
# LOAD DATA

df = pd.read_csv('data/v2merged.csv', usecols=['ga:eventAction', 'ga:eventLabel', 'ga:eventValue']) 
df = df.rename(columns={'ga:eventAction': 'id', 'ga:eventLabel': 'attempt', 'ga:eventValue': 'score'}) # rename columns 

print('data loaded')

data loaded


### Data Cleaning

In [3]:
%%time

# 1. FILTER OUT PLAYERS WITH LESS THAN 3 AND MORE THAN 200 ATTEMPTS
counts = df.groupby('id')['attempt'].count().reset_index()
ids1 = counts.drop(counts[(counts.attempt < 3) | (counts.attempt > 200)].index) # filter for n attempts 
mask = df['id'].isin(ids1.id)

# 2. FILTER OUT PLAYERS WHO SCORED 176 IN ALL ATTEMPTS (NULL SCORES)
# 176 is the score players receive when they start the game but never actually play (timer runs out before they click on a protein target) 
ids2 = df[mask].groupby('id').apply(lambda x: ~((x.score == 176.0).all()))

ids1 = ids1.set_index('id') 
ids2 = pd.DataFrame(ids2, columns=['bool176']) 
ids1 = ids1.join(ids2) 
ids1.drop(ids1[ids1.bool176==False].index, inplace=True) # drop players who scored 176 in all attempts

# 3. FILTER OUT PLAYERS WITH NaNs
nans = df[df.score.isnull()] # find NaNs
nans_array = nans.id.values
ids1 = ids1.reset_index()
ids1.drop(ids1[ids1.id.isin(nans_array)].index, inplace=True) # drop IDs with NaNs

mask = df['id'].isin(ids1.id) # update mask

CPU times: user 2min 37s, sys: 1.81 s, total: 2min 39s
Wall time: 2min 41s


In [4]:
grouped = df[mask].groupby('id') # apply mask and group by id

In [5]:
print('N OF PLAYERS AFTER MASKING: ', grouped.ngroups)

N OF PLAYERS AFTER MASKING:  460435


### Score Ratio Computation

In [6]:
# DEFINE SCORE RATIO FUNCTION

def running_best_score(sf):
    sf = sf.sort_values('attempt')
    sf['quit'] = 0
    sf.set_value(sf[sf['attempt']==sf['attempt'].max()].index, 'quit', '1') # mark quit as 1 for last game 
    sf['rolling_max'] = pd.rolling_max(sf['score'], window=len(sf), min_periods=1)
    sf['ratio'] = np.NaN
    sf['ratio'][1:] = sf['score'][1:].values / sf['rolling_max'][:-1].values    
    return sf

In [7]:
%%time

# CALC SCORE RATIOS 
# this cell takes approx. 40 mins to run (but time varies depending on sample size and pc configuration) 

outdf = grouped.apply(running_best_score)

/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: pd.rolling_max is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=4,min_periods=1,center=False).max()
  import sys
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: pd.rolling_max is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=3,min_periods=1,center=False).max()
  import sys
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: pd.rolling_max is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=8,min_periods=1,center=False).max()
  import sys
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: pd.rolling_max is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=9,min_periods=1,center=Fa

/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: pd.rolling_max is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=14,min_periods=1,center=False).max()
  import sys
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: pd.rolling_max is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=48,min_periods=1,center=False).max()
  import sys
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: pd.rolling_max is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=23,min_periods=1,center=False).max()
  import sys
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: pd.rolling_max is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=132,min_periods=1,cent

/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: pd.rolling_max is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=70,min_periods=1,center=False).max()
  import sys
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: pd.rolling_max is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=44,min_periods=1,center=False).max()
  import sys
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: pd.rolling_max is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=73,min_periods=1,center=False).max()
  import sys
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: pd.rolling_max is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=69,min_periods=1,cente

/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: pd.rolling_max is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=110,min_periods=1,center=False).max()
  import sys
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: pd.rolling_max is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=80,min_periods=1,center=False).max()
  import sys
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: pd.rolling_max is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=177,min_periods=1,center=False).max()
  import sys
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: pd.rolling_max is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=68,min_periods=1,cen

/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: pd.rolling_max is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=172,min_periods=1,center=False).max()
  import sys
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: pd.rolling_max is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=108,min_periods=1,center=False).max()
  import sys
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: pd.rolling_max is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=199,min_periods=1,center=False).max()
  import sys
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: pd.rolling_max is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=96,min_periods=1,ce

/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: pd.rolling_max is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=103,min_periods=1,center=False).max()
  import sys
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: pd.rolling_max is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=192,min_periods=1,center=False).max()
  import sys
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: pd.rolling_max is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=171,min_periods=1,center=False).max()
  import sys
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: pd.rolling_max is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=126,min_periods=1,c

CPU times: user 42min 32s, sys: 21.4 s, total: 42min 53s
Wall time: 42min 59s


### Save To CSV

In [8]:
outdf.to_csv('data/outdf.csv') # make sure your paths match